##### Part IA Experimental Engineering
# First Year Mechanics Laboratory  - Experiment 1 (long)
# Measurement of Vehicle Motion

- This experiment normally takes place in the DPO, but this year the lab is being run online. 
- You need to work through this Jupyter iPython notebook, then use the figures generated to write a lab report.
- The notebook follows the one used for the physical experiment in the DPO, agumented by videos of the experiment being conducted and matching data to analyze. If the embedded videos fail, there are direct links to panopto from the moodle site. 
- It is important to watch the videos, with sound, as they contain information you need to complete the lab.
- The notebook can be run online in google colabs (as per instructions on moodle) or downloaded and run locally in a web-browser. 
- You do NOT need to write any Python code, apart from simple editing of some values.


The Python code in the next cell will import some libraries and initialize and personalize the notebook. 

Change the watermark field to the two four-letter words provided to you by the demonstrator

then run the cell (shift+enter) and proceed.

In [ ]:
# THIS CELL SHOULD ONLY NEED TO BE RUN ONCE, AT THE BEGINNING OF THE SESSION. 
# IF YOU RESTART THE PYTHON KERNAL YOU WILL NEED TO RERUN THIS CELL

# EDIT THIS LINE

watermark='WordWord'  # Enter the two 4-letter words provided to you by the demonstrator. Default is watermark='WordWord'

# DO NOT CHANGE ANYTHING BELOW THIS LINE

#
# Import python packages
#
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import math
import time
import numpy as np
from scipy import integrate
from IPython.display import HTML
from numpy import loadtxt
%matplotlib inline
plt.rcParams.update({'font.size': 18})

#
# Initialise variables to hold the data
#
motiondata=np.array([],dtype='uint16') # Initialise a numpy array of unsigned 16-bit (two byte) integers
axlist=[]  # initialise some python lists (lists are computationally more efficient than numpy arrays when we don't know the final size)
aylist=[]
omegazlist=[]
tlist=[]
counterlist=[]

## 1. Introduction
### 1.1 Objectives

1. To learn about the operation and limitations of sensors for measuring acceleration, angular velocity, and wheel rotation angle.
2. To assess the suitability of individual sensors and combinations of sensors for determining the motion of a vehicle.
3. To apply the theory of planar kinematics of a particle.
4. To experience the use of a computer program for recording and analysing measured motion data.


### 1.2 Motivation
- Many modern powered road vehicles (cars, motorbikes, trucks) are equipped with sensors to measure the motion of the vehicle.
- The measured data can be used to actively control the vehicle (via its engine, brakes, suspension and steering) to improve stability, increase comfort, or provide autonomy.
- In this experiment the focus is on measuring the motion of a car travelling in a horizontal plane.
- The theory of particle kinematics that you learnt last term will be relevant.
- Remember that the kinematics of a particle can be expressed in cartesian, polar or intrinsic coordinates.
- In this experiment, intrinsic coordinates will be the most appropriate, see extract from Mechanics Data Book in Figure 1.
- Intrinsic coordinates involve describing the motion of a particle (or car in this case) using the speed along the path and the direction (angle) of the path.
- Note that the speed of the particle perpendicular to the path is always zero, although the acceleration perpendicular to the path may not be zero.

![intrinsic.jpg](https://github.com/CambridgeEngineering/PartIA-Mechanics-Labs/blob/main/IA_vehicle_motion/accel.jpeg?raw=true)

Figure 1. Extract from Mechanics Data Book: intrinsic coordinates for planar kinematics

### 1.3 Equipment

The experiment involves the following equipment:
- model car
- radio receiver (black box with antenna)
- USB cable
- tape measure

The model car (a 1/18 scale Ferrari 488GTB) incorporates:
- microcontroller (Arduino Nano)
- inertial measurement unit (IMU) (Bosch BNO055)
- optical rotary encoder (Bourns ENT1J-D28-L00128L)
- radio transmitter (Xbee S1 2.4GHz)
- rechargeable battery (5 V)

These items are identified in Figure 2. 

![breadboard-labels.jpg](https://github.com/CambridgeEngineering/PartIA-Mechanics-Labs/blob/main/IA_vehicle_motion/breadboard.jpeg?raw=true)
Figure 2: Equipment

- Watch the video introducing the model car in the cell below. 

- You may need to run the "code cell" to load the video before you can watch it. 

- If the video doesn't work, you can find a direct panopto link on the moodle site.



In [5]:
HTML('<iframe src="https://cambridgelectures.cloud.panopto.eu/Panopto/Pages/Embed.aspx?id=6dbe11f1-2a2e-4cf7-8e7a-acb501541be8&autoplay=false&offerviewer=true&showtitle=true&showbrand=false&start=0&interactivity=all" height="405" width="720" style="border: 1px solid #464646;" allowfullscreen allow="autoplay"></iframe>')

The first four items are described briefly in the next sections. Read these details carefully, as they will help you interpret the data from the experiment. Links to further information are provided in the Appendix.


### 1.4 Inertial measurement unit (IMU)
- The IMU is a System in Package (SiP), measuring 5.2mm x 3.8mm x 1.1mm.
- The IMU is located on the longitudinal centre line of the car, above the rear axle.
- It measures translational acceleration and angular velocity in three orthogonal axes labelled x, y, z.
- The x,y,z axes are fixed in the car and are oriented as shown in Figure 2.
- The IMU package includes a microcontroller which can self-calibrate the sensors and estimate absolute angular orientation, but this experiment only uses the raw measurements of longitudinal and lateral accelerations in a horizontal plane (x and y axes) and angular velocity about a vertical axis (z) also known as yaw velocity.

### 1.5 Optical rotary encoder
- The left-hand (LH) rear wheel of the vehicle is coupled to an optical rotary encoder, see Figure 2. (Left-hand when looking in the positive x direction.) 
- The encoder contains: a slotted disc that rotates with the wheel; two stationary phototransistors (light sensitive switches) on one side of the disc; and a stationary source of light on the other side of the disc.
- When the slotted disc rotates, the light falling on the phototransistors is interrupted so that the output of each phototransistor is a square wave; the two square waves are 90 degrees out of phase with each other.
- The microcontroller program counts the rising pulse from one of the phototransistors and uses the corresponding state of the other phototransistor to determine the direction of rotation.
- The count has an integer value and is initialised to zero when the microcontroller is turned on.
- The  count increases or decreases according to whether the wheel is rolling forwards or backwards.
- One whole rotation of the wheel changes the count by 128.
- The maximum value of the count is 32767 and the minimum value is -32768.
- Provided that the wheel does not slip on the road, the count indicates the distance travelled by the wheel.
- IMPORTANT NOTE: if the car is not following a straight path the distance travelled by the LH wheel may not be the same as the distance travelled by the IMU, because there is a lateral offset between the LH wheel and the IMU.

![encoder-labels.jpg](https://github.com/CambridgeEngineering/PartIA-Mechanics-Labs/blob/main/IA_vehicle_motion/encoder.jpeg?raw=true)

Figure 3. Optical rotary encoder attached to left-hand rear wheel

### 1.6 Arduino Nano microcontroller
- A microcontroller is a computer on a single integrated circuit, containing one or more CPUs, memory and input/output peripherals. 
- 'Arduino' is an open-source electronics platform based on easy-to-use hardware and software. 
- The Arduino Nano is a board that includes an ATmega328P microcontroller (Microchip Technology Inc.). 
- The board contains everything needed to support the microcontroller.
- In this experiment the board is programmed to: read longitudinal and lateral acceleration and yaw velocity data from the IMU; count the pulses from the encoder; and continually send data to the personal computer via the radio link or USB cable.

### 1.7 Transmitting and receiving data
The Arduino transmits the sensor data to a radio receiver that attatches to the computer via USB. 


### 1.8 Load first data set
- For each part of the experiment, you will watch a video of the model car being moved, and then load the resulting dataset from github by running a code cell of the form:

[axlist, aylist, omegazlist, counterlist,tlist] = loadtxt('https://raw.githubusercontent.com/CambridgeEngineering/PartIA-Mechanics-Labs/main/IA_vehicle_motion/data.csv', delimiter=',').tolist()

- This cell loads a time series of  x-acceleration, y-acceleartion, yaw-velocities, wheel-counts to the variables  axlist, aylist, omegazlist, counterlist. The variable tlist contains the corrosponding list of times.

- N.B. when you load a data-set, it will overwrite the previous one. You can always get back the earlier data set by loading it again.

- We start by examining the data from a few basic trial motions of the car. This will help us understand what the sensors measure.

- Use the next two cells to watch the video of the motions, and then load the data.





In [6]:
#Video Cell. You may need to run this cell to load the video before you can play it
HTML('<iframe src="https://cambridgelectures.cloud.panopto.eu/Panopto/Pages/Embed.aspx?id=d3df8c31-b878-4cd3-a98c-acb501550ddd&autoplay=false&offerviewer=true&showtitle=true&showbrand=false&start=0&interactivity=all" height="405" width="720" style="border: 1px solid #464646;" allowfullscreen allow="autoplay"></iframe>')

In [ ]:
#Data load cell
[axlist, aylist, omegazlist, counterlist,tlist] = loadtxt('https://raw.githubusercontent.com/CambridgeEngineering/PartIA-Mechanics-Labs/main/IA_vehicle_motion/datamotions.csv', delimiter=',').tolist()
print('Basic motion data loaded')

### 1.9 Examine the data
- The cell below plots the data from the four sensors. 

- The plot is not used in the report. 

- You can run this cell at any time to examine the currently loaded data. It is a good idea to do this each time you load new data, to check the data really has loaded and get a first look at what it contains.

- Run the cell, and verify that you can identify each motion from the video in the plots. 



In [ ]:
# You can run this cell any time to plot the currently loaded raw data
# It creates simple plots of the lateral accelearation, yaw velocity, wheel encoder count and longitudinal acceleation.

t=np.array(tlist)   # convert lists to numpy arrays
ax=np.array(axlist)  
ay=np.array(aylist)
omegaz=np.array(omegazlist) 
counter=np.array(counterlist)
    
t=t-t[0] # subtract first element of the array from the whole array so that the plot starts at 0 s.
counter=counter-counter[0] # subtract first element of the array from the whole array so that the counter starts at 0.

fig11=plt.figure(11,figsize=(19,12)) 
plt.subplot(221)
plt.title('Lateral acceleration')
plt.ylabel(r'$a_y\ /\ \rm{m \ s^{-2}}$')
plt.plot(t,ay,marker='o', markersize=3, color="red") # list of line objects
plt.subplot(222)
plt.title('Wheel encoder count')
plt.ylabel(r'$n$')
plt.plot(t,counter,marker='o', markersize=3, color="green")
plt.subplot(223)
plt.title('Yaw velocity')
plt.xlabel(r'time $t\ /\ \rm{s}$')
plt.ylabel(r'$\omega_z\ /\ \rm{rad \ s^{-1}}$')
plt.plot(t,omegaz, marker='o', markersize=3, color="blue")
plt.subplot(224)
plt.title('Longitudinal acceleration')
plt.xlabel(r'time $t\ /\ \rm{s}$')
plt.ylabel(r'$a_x\ /\ \rm{m \ s^{-2}}$')
plt.plot(t,ax, marker='o', markersize=3, color="orange")
fig11.text(0.5,0.5,watermark,fontsize=140,color='gray',ha='center',va='center',rotation=45,alpha=0.15)
plt.show()

Confirm that you can identify each motion from the video in the plots. 

It will help to consider the following points about the sensor data.

- When the car is stationary but vertical, a non-zero acceleration is recorded. This is because the IMU does not measure acceleration directly; it measures the displacement of a mass at the end of an elastic cantilever. The force due to gravity as well as acceleration cause displacement of the mass.

- The effect of gravity can also be seen on the recorded accelerations when the car is pointed downwards, and when the car is rolled sideways through ninety degrees.

- Yaw velocity is only observed when the car is rotating around the z axis. (This isn't in the current data set, but a new better one will be made soon)

- The wheel count only changes when the wheel turns - i.e. when the car is pushed along the bench.

- The wheel encoder count is always an integer value, and an increasing count corresponds to forward motion.

## 2. Stationary car

- In this section you will: examine in more detail the characteristics of the recorded signals; generate Figure 1; and answer Question 1.
- Data was captured for about 5 seconds while the car was stationary on the desk. 
- Run the next three cells to watch the video of the experiment, load the data, and produce Figure 1 using the recorded data.

In [7]:
#Video Cell. You may need to run this cell to load the video before you can play it
HTML('<iframe src="https://cambridgelectures.cloud.panopto.eu/Panopto/Pages/Embed.aspx?id=3a620845-316e-4c78-bef3-acb10133b4ba&autoplay=false&offerviewer=true&showtitle=true&showbrand=false&start=0&interactivity=all" height="405" width="720" style="border: 1px solid #464646;" allowfullscreen allow="autoplay"></iframe>')

In [ ]:
#Data load cell
[axlist, aylist, omegazlist, counterlist,tlist] = loadtxt('https://raw.githubusercontent.com/CambridgeEngineering/PartIA-Mechanics-Labs/main/IA_vehicle_motion/datasta.csv', delimiter=',').tolist()
print('Stationary data loaded')

In [ ]:
# Plot figure 1.

t=np.array(tlist)   # convert lists to numpy arrays
ax=np.array(axlist)  
ay=np.array(aylist)
omegaz=np.array(omegazlist) 
counter=np.array(counterlist)
    
t=t-t[0] # subtract first element of the array from the whole array so that the plot starts at 0 s.
counter=counter-counter[0] # subtract first element of the array from the whole array so that the counter starts at 0.

fig1=plt.figure(1,figsize=(19,12)) 
plt.subplot(221)
plt.title('Lateral acceleration')
plt.ylabel(r'$a_y\ /\ \rm{m \ s^{-2}}$')
plt.plot(t,ay,marker='o', markersize=3, color="red") # list of line objects
plt.subplot(222)
plt.title('Wheel encoder count')
plt.ylabel(r'$n$')
plt.plot(t,counter,marker='o', markersize=3, color="green")
plt.subplot(223)
plt.title('Yaw velocity')
plt.xlabel(r'time $t\ /\ \rm{s}$')
plt.ylabel(r'$\omega_z\ /\ \rm{rad \ s^{-1}}$')
plt.plot(t,omegaz, marker='o', markersize=3, color="blue")
plt.subplot(224)
plt.title('Longitudinal acceleration')
plt.xlabel(r'time $t\ /\ \rm{s}$')
plt.ylabel(r'$a_x\ /\ \rm{m \ s^{-2}}$')
plt.plot(t,ax, marker='o', markersize=3, color="orange")
fig1.text(0.5,0.5,watermark,fontsize=140,color='gray',ha='center',va='center',rotation=45,alpha=0.15)
plt.show()

- The data have been obtained by sampling and quantizing the real physical motion of the car.
- Sampling is the process of measuring a continuous signal at an instant in time (in this case every 50 ms, or 20 Hz).
- Quantization is the process of converting the value of the original continuous (analogue) signal at the sampling time into one of a finite set of discrete (digital) values.
- The combined process of sampling and quantization is known as analogue to digital conversion (A to D).
- The quantized values are initially generated as integers (not floating point numbers), then multiplied by a scaling or calibration factor to specify the signal in appropriate physical units.
- The sampled and quantized data are plotted as small dots, and to aid visualisation lines are plotted between the dots.
- Consider the following questions and write your answers in your lab notebook (each person in the lab group should record their own set of answers).

#### Q1a. Can you confirm that the sampling rate is 20 Hz? How?

#### Q1b. For each of the four signals:
> - What is the quantization step size (smallest non-zero change in value)?
> - What is the approximate mean value of the signal? Is the mean value what you expect? Why?
> - What is the approximate peak-to-peak value of the signal? Is the peak-to-peak value what you expect? Why?


- When you have answered the questions, save Figure 1 in pdf and png format by running the next cell.
- If you are running on colabs, the files will only be saved on the remote google server, and will be deleted if the server disconects (for example, if the notebook is idle for too long). Therefore, you should then immediately run the next cell which will download the files to your local machine via the browser. 
- If you are running python locally, then the save comand will save the file somewhere on your local system - probably in the same directory as the python notebook. If you can't find it, try searching your disk for "fig1pdf.pdf". You cannot run the download comand, as it is specific to colabs.
 

In [ ]:
# Use this cell to save figure 1
if 'fig1' in globals():
  fig1.savefig("fig1pdf.pdf")
  fig1.savefig("fig1png.png")
  print('Figure 1 saved')
else: print('Error: Figure not defined')

In [ ]:
# Google colabs only: then use this cell to download figure 1
# You may get a browser pop up asking for permission to download (multiple) files - if so, you should grant permission.
from google.colab import files
files.download('fig1pdf.pdf')
files.download('fig1png.png')

## 3. Straight-line motion 

- In this section you will: use the wheel encoder count and the longitudinal acceleration measured by the IMU to determine the straight-line motion of the car; generate Figures 2 and 3; and answer Questions 2 and 3.

### 3.1 Calibrate the optical rotary encoder
- Before using the encoder to measure distance travelled, the encoder needs to be calibrated.
- To do this, we move the car a known distance of about 1.2m.
- Watch the video and load the data by running the next two cells.





In [8]:
#Video Cell. You may need to run this cell to load the video before you can play it
HTML('<iframe src="https://cambridgelectures.cloud.panopto.eu/Panopto/Pages/Embed.aspx?id=34eee688-b4c5-4901-8b3e-acb101349b7a&autoplay=false&offerviewer=true&showtitle=true&showbrand=false&start=0&interactivity=all" height="405" width="720" style="border: 1px solid #464646;" allowfullscreen allow="autoplay"></iframe>')

In [ ]:
#Data load cell
[axlist, aylist, omegazlist, counterlist,tlist] = loadtxt('https://raw.githubusercontent.com/CambridgeEngineering/PartIA-Mechanics-Labs/main/IA_vehicle_motion/datastraight.csv', delimiter=',').tolist()
print('Straight slow data loaded')

- The cell below contains code to generate Figure 2.
- Edit the first line of code to specify the distance 'tapedistance' in metres that the car moved in the video. (Do not edit the value of 'wheeldia' at this stage.)
- The rest of the code copies the loaded data to numpy arrays, then the wheel encoder data is converted to distance and plotted against time.
- The conversion from count to distance involves dividing by the pulses per revolution of the encoder ('ppr' in the code = 128), multiplying by pi and multiplying by the diameter of the wheel ('wheeldia' in the code).
- Run the cell and compare the tape measure distance (orange dashed line) to the distance determined from the wheel encoder count (blue solid line).

In [ ]:
#CODE TO MAKE FIGURE 2  

# EDIT THE NEXT LINE TO SPECIFY THE DISTANCE IN m MEASURED USING THE TAPE MEASURE (default is 1.000)

tapedistance=1.000

# EDIT THE NEXT LINE TO SPECIFY THE DIAMETER OF THE REAR WHEEL IN m (default is 0.0400)

wheeldia=0.04


# DO NOT CHANGE ANYTHING BELOW THIS LINE

plt.close(0) # close figure 0 if current


t=np.array(tlist)   # convert lists to numpy arrays
ax=np.array(axlist)  
ay=np.array(aylist)
omegaz=np.array(omegazlist) 
counter=np.array(counterlist)

    
t=t-t[0] # subtract first element of the array from the whole array so that the plot starts at 0 s.
counter=counter-counter[0] # subtract first element of the array from the whole array so that the counter starts at 0.

ppr=128 # pulses per revolution of the encoder
scount=(counter/ppr)*(wheeldia*np.pi) # convert the count to distance in metres

fig2=plt.figure(2,figsize=(9.5,6)) 
plt.title('Wheel encoder distance (wheeldia=%.4f m)' %wheeldia)
plt.xlabel(r'time $t\ /\ \rm{s}$')
plt.ylabel(r'distance $s\ /\ \rm{m}$')
plt.plot(t,scount)
plt.plot(t,tapedistance*np.ones_like(t),linestyle='--')
fig2.text(0.5,0.5,watermark,fontsize=70,color='gray',ha='center',va='center',rotation=45,alpha=0.15)
plt.show()

- Are the tape measure distance (orange dashed line) and the final distance determined from the wheel encoder count (blue solid line) sufficiently close? The lines should be on top of each other.
- If not, edit the value of 'wheeldia' in the second line of code to get agreement and thus to calibrate the wheel encoder count. Then answer Question 2.

#### Q2. What value of 'wheeldia' did you use? Is this consistent with the diameter seen in the image below?

![diameter.jpg](https://github.com/CambridgeEngineering/PartIA-Mechanics-Labs/blob/main/IA_vehicle_motion/diameter.jpg?raw=true)

- When you have done this, save Figure 2 to your filespace by running the cell below (existing files with the same name will be overwritten without warning).

In [ ]:
# Use this cell to save figure 2
if 'fig2' in globals():
  fig2.savefig("fig2pdf.pdf")
  fig2.savefig("fig2png.png")
  print('Figure 2 saved')
else: print('Error: Figure not defined')

In [ ]:
# Google colabs only: then use this cell to download figure 2
# You may get a browser pop up asking for permission to download (multiple) files - if so, you should grant permission.
from google.colab import files
files.download('fig2pdf.pdf')
files.download('fig2png.png')

### 3.2 Compare the measurements of the IMU and the wheel encoder

- A practical difficulty with using wheel rotation to measure distance travelled is that the method relies on the wheel not slipping on the road surface.
- Whenever a wheel is driven or braked, slipping will occur.
- It's not a significant problem in this experiment because the wheel is free to roll, not driven or braked, but on real vehicles the slippping can be significant. 


- An alternative way to measure distance travelled is to measure the longitudinal acceleration using the IMU (x direction) and then integrate twice to estimate distance.
- This works better if the acceleration/deceleration is large, so we now repeat the straight line experiment, but moving the car faster (aiming to achieve a maximum acceleration of about 5 m/s^)- During this experiment, it is vital that the car starts and finishes at rest.
- Watch the video and load the data by running the next two cells.



In [9]:
#Video Cell. You may need to run this cell to load the video before you can play it
HTML('<iframe src="https://cambridgelectures.cloud.panopto.eu/Panopto/Pages/Embed.aspx?id=ceb5b3eb-a0b0-4b29-9a10-acb10134badb&autoplay=false&offerviewer=true&showtitle=true&showbrand=false&start=0&interactivity=all" height="405" width="720" style="border: 1px solid #464646;" allowfullscreen allow="autoplay"></iframe>')

In [ ]:
#Data load cell
[axlist, aylist, omegazlist, counterlist,tlist] = loadtxt('https://raw.githubusercontent.com/CambridgeEngineering/PartIA-Mechanics-Labs/main/IA_vehicle_motion/datastraightfast.csv', delimiter=',').tolist()
print('Straight fast data loaded')


- The code in the next cell numerically integrates the measured longitudinal acceleration data using the trapezium rule to estimate longitudinal velocity. The initial velocity is set to zero.
- The longitudinal velocity is then integrated to distance. The initial distance is set to zero.
- A constant 'axoffset' can be added to the acceleration before it is integrated. The constant is intially set to zero in the code.


- The code also numerically differentiates the measured encoder data, using numpy's 'gradient' function.
- Differentiating once gives longitudinal velocity, differentiating again gives longitudinal acceleration (at the left-hand wheel)
- The wheel encoder data is calibrated using the value of wheeldia determined in Section 3.1.


- The longitudinal acceleration, velocity and distance determined from the IMU and from the encoder are then plotted in separate graphs but with a common time axis.
- Run the cell initially with axoffset=0 (the default value).

In [ ]:
# EDIT THE VALUE OF axoffset TO COMPENSATE OFFSET IN THE ACCELERATION SIGNAL

axoffset = 0.0



plt.close(0) # close figure 0 if current


t=np.array(tlist)   # convert lists to numpy arrays
ax=np.array(axlist)  
ay=np.array(aylist)
omegaz=np.array(omegazlist) 
counter=np.array(counterlist)

    
t=t-t[0] # subtract first element of the array from the whole array so that the plot starts at 0 s.
counter=counter-counter[0] # subtract first element of the array from the whole array so that the counter starts at 0.

axo = ax+axoffset
vx = integrate.cumtrapz(axo, t, initial=0)
sx = integrate.cumtrapz(vx, t, initial=0)

ppr=128 # pulses per revolution of the encoder
sxcount=(counter/ppr)*(wheeldia*np.pi) # convert the count to distance in metres
vxcount=np.gradient(sxcount,t) # differentiate to find velocity
axcount=np.gradient(vxcount,t) # differentiate again to find acceleration

fig3=plt.figure(3,figsize=(19,12)) 
plt.subplot(311)
plt.title('IMU (solid) and encoder (dash). axoffset=%.2f m/s^2' %axoffset)
plt.ylabel(r'long. accn. $/\ \rm{m \ s^{-2}}$')
plt.plot(t,axo) 
plt.plot(t,axcount,linestyle='--') 
plt.subplot(312)
plt.ylabel(r'long. vel. $/\ \rm{m \ s^{-1}}$')
plt.plot(t,vx)
plt.plot(t,vxcount,linestyle='--')
plt.subplot(313)
plt.xlabel(r'time $t\ /\ \rm{s}$')
plt.ylabel(r'distance $/\ \rm{m}$')
plt.plot(t,sx)
plt.plot(t,sxcount,linestyle='--')
fig3.text(0.5,0.5,watermark,fontsize=70,color='gray',ha='center',va='center',rotation=45,alpha=0.15)
plt.show()

#### Q3a. Is the data as you expect? Are there any discrepancies between the two sets of data, and what might be the causes?

- The longitudinal velocity at the end of the run should be zero (assuming you stopped recording after the vehicle stopped moving).
- It is likely that this is not the case for the velocity signal determined from the IMU (solid line).
- The reason is due mainly to the small non-zero acceleration measured when the car is stationary (see your findings in Section 2, Figure 1 and Question 1b).
- This constant offset results in drift of the calculated velocity and distance away from the true values.
- If it is assumed that the offset is constant, then it can be compensated by adding a constant to the measured acceleration before integrating to velocity and displacement.
- Re-run the cell with a non-zero value of 'axoffset' (edit the first line in the cell), to bring the final velocity determined from the IMU (solid line) close to zero.

#### Q3b. What value of offset did you use? Was it effective in eliminating the drift of the velocity and displacement signals determined from the IMU's acceleration signal? Are there any remaining discrepancies between the two sets of data?

- Offsets in the outputs of sensors are very common (and not just inertial sensors).
- The offset can vary with time, temperature and other factors.
- The processing of measured data must often account for the presence of offsets.
- In the case of accelerometers, unwanted offsets can also arise due to the gravitational field.
- In vehicles this occurs when operating on a non-horizontal road surface.
- The IMU's microcontroller can compensate for sensor offsets and gravitational field, and can also calibrate the acceleration and angular velocity sensors, but this facility is not used in the present experiment.


- When you have answered Question 3, save Figure 3 to your filespace by running the cell below (existing files with the same name will be overwritten without warning).

In [ ]:
# Use this cell to save figure 3
if 'fig1' in globals():
  fig3.savefig("fig3pdf.pdf")
  fig3.savefig("fig3png.png")
  print('Figure 3 saved')
else: print('Error: Figure not defined')

In [ ]:
# Google colabs only: then use this cell to download figure 3
# You may get a browser pop up asking for permission to download (multiple) files - if so, you should grant permission.
from google.colab import files
files.download('fig3pdf.pdf')
files.download('fig3png.png')

## 4. Circular motion
- In this section you will: use three different combinations of the signals to measure the instantaneous path radius of the car; generate Figures 4 to 8; answer Questions 4 to 8.

### 4.1 Calibration of yaw velocity measured by IMU

- Before proceeding further, the yaw velocity output of the IMU will be calibrated.
- To do this, we gather data while pushing the car in a complete circle, so we know it has turned 360 degrees. 
- During the motion, we try to keep the wheels steered as far as they will go to the left or the right, to achieve a constant radius.
- We also try to ensure that the car is pointing in exactly the same direction at the beginning and the end, and aim for a lateral acceleration of about 3m/s^2 without any sideways sliding. 
- We also include a coupole of seconds of stationary data at the start and end of the run.
- Watch the video and load the data by running the next two cells.


In [10]:
#Video Cell. You may need to run this cell to load the video before you can play it
HTML('<iframe src="https://cambridgelectures.cloud.panopto.eu/Panopto/Pages/Embed.aspx?id=1c99011b-5631-497a-a502-acb200ac7f65&autoplay=false&offerviewer=true&showtitle=true&showbrand=false&start=0&interactivity=all" height="405" width="720" style="border: 1px solid #464646;" allowfullscreen allow="autoplay"></iframe>')

In [ ]:
#Data load cell
[axlist, aylist, omegazlist, counterlist,tlist] = loadtxt('https://raw.githubusercontent.com/CambridgeEngineering/PartIA-Mechanics-Labs/main/IA_vehicle_motion/datacircle1.csv', delimiter=',').tolist()
print('Circle motion data loaded')

- The next cell numerically integrates the measured yaw velocity (plotted in the top graph) to calculate yaw angle (plotted in the middle graph). The initial value is set to zero.
- The calculated angle at the end of the run is compared with the known angle ($2\pi n_{circles}$ where $n_{circles}$ is the number of complete circles.
- Edit the first line of the cell if the car completed two or more complete circles. (Do not change the value of 'omegazscaling' at this stage.)


- Run the cell 

In [ ]:
ncircles=1           # EDIT THIS TO SPECIFY THE NUMBER OF COMPLETE CIRCLES (default is 1)
omegazscaling=1.000   # EDIT THIS TO SCALE THE YAW VELOCITY OUTPUT OF THE IMU (default is 1.00)



# DO NOT CHANGE ANYTHING BELOW THIS LINE

plt.close(0) # close figure 0 if current

t=np.array(tlist)   # convert lists to numpy arrays
ax=np.array(axlist)  
ay=np.array(aylist)
omegaz=np.array(omegazlist) 
counter=np.array(counterlist)


t=t-t[0] # subtract first element of the array from the whole array so that the plot starts at 0 s.
counter=counter-counter[0] # subtract first element of the array from the whole array so that the counter starts at 0.

ppr=128 # pulses per revolution of the encoder
sxcount=(counter/ppr)*(wheeldia*np.pi) # convert the count to distance in metres
omegazscaled=omegaz*omegazscaling
thetaz = integrate.cumtrapz(omegazscaled, t, initial=0) # integrate omegaz to find angular displacement thetaz
averagerad=sxcount[-1]/thetaz[-1] # use the final values of distance and angle for an average path radius

fig4=plt.figure(4,figsize=(19,12)) 
plt.subplot(311)
plt.title('omegazscaling=%.2f, average path radius (dashed)=%.4f m,' %(omegazscaling, averagerad))
plt.ylabel(r'yaw vel. $/\ \rm{rad \ s^{-1}}$')
plt.plot(t,omegazscaled)
plt.subplot(312)
plt.ylabel(r'yaw angle $/\ \rm{rad}$')
plt.plot(t,thetaz)
plt.plot(t,np.sign(averagerad)*ncircles*2*np.pi*np.ones_like(t),linestyle='--')
plt.subplot(313)
plt.xlabel(r'time $t\ /\ \rm{s}$')
plt.ylabel(r'distance $/\ \rm{m}$')
plt.plot(t,sxcount)
fig4.text(0.5,0.5,watermark,fontsize=140,color='gray',ha='center',va='center',rotation=45,alpha=0.15)
plt.show()

- Inspect the top graph and confirm that the yaw velocity at the beginning and end of the run is zero. (The yaw velocity sensor is less susceptible to offsets than the acceleration sensors therefore it is not necessary to apply a compensating offset before integrating the yaw velocity to yaw angle.)


- Inspect the middle graph. The solid line is the yaw angle calculated by integrating the measured yaw velocity.
- The dashed horizontal line is $2\pi n_{circles}$ radians, the actual yaw angle at the end of the path (assuming that the car completed exactly a whole number of circles)
- If the final values of the solid and dashed lines are different then a scaling (multiplying) factor must be applied to the measured yaw velocity to correct the calibration of the IMU.
- If necessary, adjust the value of 'omegazscaling' in the second line of the cell and re-run the cell.
- Repeat until the difference is negligible.
- The value of omegazscaling used is printed in the title of the figure.


- The bottom graph shows the distance travelled, determined from the calibrated encoder signal (using the value of wheeldia found in Section 3.1).
- If it is assumed that the path of the car was circular, an average value of path radius can be calculated by dividing the final yaw angle travelled by the final distance travelled.
- This radius is printed in the title of the figure.

#### Q4. Watch the video in the cell below, where we use a tape measure to estimate the radius of the path. Is this estimate consistent with the calculated average value? Why? (Remember that the encoder measures the distance travelled by the left hand rear wheel.)



In [11]:
#Video Cell. You may need to run this cell to load the video before you can play it
HTML('<iframe src="https://cambridgelectures.cloud.panopto.eu/Panopto/Pages/Embed.aspx?id=8802a992-dc44-44ab-8777-acb200ab50ad&autoplay=false&offerviewer=true&showtitle=true&showbrand=false&start=0&interactivity=all" height="405" width="720" style="border: 1px solid #464646;" allowfullscreen allow="autoplay"></iframe>')

- When you have answered Question 4 run the cell below to save Figure 4 (existing files with the same name will be overwritten without warning).

In [ ]:
# Use this cell to save figure 4
if 'fig1' in globals():
  fig4.savefig("fig4pdf.pdf")
  fig4.savefig("fig4png.png")
  print('Figure 4 saved')
else: print('Error: Figure not defined')

In [ ]:
# Google colabs only: then use this cell to download figure 4
# You may get a browser pop up asking for permission to download (multiple) files - if so, you should grant permission.
from google.colab import files
files.download('fig4pdf.pdf')
files.download('fig4png.png')

### 4.2 Estimate radius of path using encoder and yaw velocity

- Now that the yaw velocity is calibrated, a time history of the instantaneous radius $r$ of the vehicle's path can be obtained from the longitudinal velocity $v_x$ and yaw velocity $\omega_z$:

$$ r=\frac{v_x}{\omega_z}$$

- As you saw in Section 3.2, the longitudinal velocity $v_x$ can be obtained by differentiating the displacement signal from the wheel encoder.
- The next cell plots three graphs: the longitudinal velocity $v_x$ determined from the encoder; the yaw velocity $\omega_z$ measured by the IMU; and the radius $ r=\frac{v_x}{\omega_z}$.
- The bottom graph also shows the average path radius calculated previously from the final values of yaw angle and distance. 
- Note that some values of $r$ might not be plotted if there is a divide by zero condition.
- Run the cell.

In [ ]:
ymax=0.5    # EDIT THIS TO SPECIFY THE MAXIMUM RADIUS IN m ON THE VERTICAL AXIS (default is 0.5)
ymin=-0.5   # EDIT THIS TO SPECIFY THE MINIMUM RADIUS IN m ON THE VERTICAL AXIS (default is -0.5)

# DO NOT CHANGE ANYTHING BELOW THIS LINE

plt.close(0) # close figure 0 if current
np.warnings.filterwarnings('ignore') # suppresses divide by zero warnings from np.divide
vxcount=np.gradient(sxcount,t) # differentiate distance to velocity
r=np.divide(vxcount,omegazscaled) # calculate path radius

fig5=plt.figure(5,figsize=(9.5,6)) 
plt.subplot(311)
plt.title('omegazscaling=%.2f, average path radius (dashed)=%.4f m,' %(omegazscaling, averagerad))
plt.ylabel(r'long. vel. $/\ \rm{m \ s^{-1}}$')
plt.plot(t,vxcount)
plt.subplot(312)
plt.ylabel(r'yaw vel. $/\ \rm{rad \ s^{-1}}$')
plt.plot(t,omegazscaled)
plt.subplot(313)
plt.ylim(ymin,ymax)
plt.ylabel(r'radius $/\ \rm{m}$')
plt.xlabel(r'time $t\ /\ \rm{s}$')          
plt.plot(t,r)
plt.plot(t,averagerad*np.ones_like(t),linestyle='--')
fig5.text(0.5,0.5,watermark,fontsize=70,color='gray',ha='center',va='center',rotation=45,alpha=0.15)
plt.show()

#### Q5. Comment on the whole time history of radius (solid line) in the bottom graph, and compare it to the average path radius calculated in Section 4.1 (dashed line).
> #### In particular, comment on regions where there is agreement and where there is discrepancy.
> #### You may need to adjust the range of the vertical axis in the last graph; edit the ymax and ymin values in the cell.

- When you have answered Question 5 run the cell below to save Figure 5 (existing files with the same name will be overwritten without warning).

In [ ]:
# Use this cell to save figure 5
if 'fig1' in globals():
  fig5.savefig("fig5pdf.pdf")
  fig5.savefig("fig5png.png")
  print('Figure 5 saved')
else: print('Error: Figure not defined')

In [ ]:
# Google colabs only: then use this cell to download figure 5
# You may get a browser pop up asking for permission to download (multiple) files - if so, you should grant permission.
from google.colab import files
files.download('fig5pdf.pdf')
files.download('fig5png.png')

### 4.3 Estimate radius of path using encoder and lateral acceleration

- The data book diagram of intrinsic coordinates reveals that path radius can also be calculated using longitudinal velocity $v_x$ and lateral acceleration $a_y$:

$$ r=\frac{v_x^2}{a_y}$$


- The next cell plots three graphs: the longitudinal velocity $v_x$ (determined from the encoder); the lateral acceleration $a_y$ (from the IMU); and the radius $ r=\frac{v_x^2}{a_y}$.
- The dashed line in the bottom graph is the average radius calculated previously in Section 4.1.
- Note that some values of $r$ might not be plotted if there is a divide by zero condition. 
- Run the cell. (Do not change the values of 'ymax', 'ymin' or 'ayoffset' at this stage.)

In [ ]:
ymax=0.5    # EDIT THIS TO SPECIFY THE MAXIMUM RADIUS ON THE VERTICAL AXIS (default is 0.5 m)
ymin=-0.5   # EDIT THIS TO SPECIFY THE MINIMUM RADIUS ON THE VERTICAL AXIS (default is -0.5 m)
ayoffset=0.00  # EDIT THIS VALUE TO COMPENSATE OFFSET IN THE LATERAL ACCELERATION SIGNAL (default is 0.00 m/s/s)

# DO NOT CHANGE ANYTHING BELOW THIS LINE

plt.close(0) # close figure 0 if current
np.warnings.filterwarnings('ignore') # suppresses divide by zero warnings from np.divide
ayo = ay+ayoffset
r=np.divide(np.square(vxcount),ayo) # calculate path radius

fig6=plt.figure(6,figsize=(9.5,6)) 
plt.subplot(311)
plt.title('omegazscaling=%.2f, ayoffset=%.2f m/s/s, average path radius (dashed)=%.4f m,' %(omegazscaling, ayoffset, averagerad))
plt.ylabel(r'long. vel. $/\ \rm{m \ s^{-1}}$')
plt.plot(t,vxcount)
plt.subplot(312)
plt.ylabel(r'lat. accn. $a_y\ /\ \rm{m \ s^{-2}}$')
plt.plot(t,ayo)
plt.subplot(313)
plt.ylim(ymin,ymax)
plt.ylabel(r'radius $/\ \rm{m}$')
plt.xlabel(r'time $t\ /\ \rm{s}$')           
plt.plot(t,r)
plt.plot(t,averagerad*np.ones_like(t),linestyle='--')
fig6.text(0.5,0.5,watermark,fontsize=70,color='gray',ha='center',va='center',rotation=45,alpha=0.15)
plt.show()

- A difficulty with using lateral acceleration measured by the IMU is the possible presence of an offset.
- Inspect the middle graph and check that lateral acceleration is zero at the start and end, when the car is stationary.
- If it is not zero, edit the value of 'ayoffset' in the third line of the cell above and re-run the cell.
- When you are happy that the offset of the lateral acceleration has been compensated answer Question 6.

#### Q6. Comment on the whole time history of radius (solid line) in the bottom graph, and compare it to the average path radius (dashed line) calculated in Section 4.1.
> #### In particular, comment on regions where there is agreement and where there is discrepancy, and compare to the bottom graph of Figure 5.
> #### You may need to adjust the range of the vertical axis in the last graph; edit the ymax and ymin values in the cell.
> #### Remember that the encoder measures the distance travelled by the left hand rear wheel, and that the IMU is located on the longitudinal centreline of the car; what effect does this have on the calculated path radius? 

- When you have answered Question 6 run the cell below to save Figure 6 (existing files with the same name will be overwritten without warning).

In [ ]:
# Use this cell to save figure 6
if 'fig6' in globals():
  fig6.savefig("fig6pdf.pdf")
  fig6.savefig("fig6png.png")
  print('Figure 6 saved')
else: print('Error: Figure not defined')

In [ ]:
# Google colabs only: then use this cell to download figure 6
# You may get a browser pop up asking for permission to download (multiple) files - if so, you should grant permission.
from google.colab import files
files.download('fig6pdf.pdf')
files.download('fig6png.png')

### 4.4  Estimate path radius using longitudinal acceleration and yaw velocity

- In Section 4.2 a time history of the instantaneous radius $r$ of the vehicle's path was estimated using the longitudinal velocity $v_x$ derived from the measured encoder signal, and using the measured yaw velocity $\omega_z$ from the IMU:

$$ r=\frac{v_x}{\omega_z}$$

- In this section a similar calculation of path radius is performed, but the longitudinal velocity $v_x$ is determined instead by integrating the longitudinal acceleration (from the IMU).
- The first step is to determine the acceleration offset required to ensure zero velocity when the vehicle is stationary at the end of the run.
- Run the next cell to integrate longitudal acceleration (from the IMU) and plot a graph of the longitudinal velocity $v_x$. Do not change the value of 'axoffset' at this stage.

In [ ]:
axoffset = 0.00  # EDIT THIS VALUE TO COMPENSATE OFFSET IN THE LONGITUDINAL ACCELERATION SIGNAL (default is 0.00 m/s/s)

# DO NOT CHANGE ANYTHING BELOW THIS LINE

axo = ax+axoffset
vx = integrate.cumtrapz(axo, t, initial=0)

fig7=plt.figure(7,figsize=(9.5,6)) 
plt.subplot(211)
plt.title('axoffset=%.2f m/s/s' %axoffset)
plt.ylabel(r'$ long. accn. a_x\ /\ \rm{m \ s^{-2}}$')
plt.plot(t,axo)
plt.subplot(212)
plt.ylabel(r'long. vel. $/\ \rm{m \ s^{-1}}$')
plt.xlabel(r'time $t\ /\ \rm{s}$')  
plt.plot(t,vx)
fig7.text(0.5,0.5,watermark,fontsize=70,color='gray',ha='center',va='center',rotation=45,alpha=0.15)
plt.show()

- Examine the final longitudinal velocity in the bottom graph. Now edit the value of 'axoffset' in the cell above to bring the final velocity to zero.

#### Q7. What value of offset did you use? Is it different to the value used for Figure 3 and Question 3b? Why?

- Run the next cell to save Figure 7 (existing files with the same name will be overwritten without warning).

In [ ]:
# Use this cell to save figure 7
if 'fig7' in globals():
  fig7.savefig("fig7pdf.pdf")
  fig7.savefig("fig7png.png")
  print('Figure 7 saved')
else: print('Error: Figure not defined')

In [ ]:
# Google colabs only: then use this cell to download figure 7
# You may get a browser pop up asking for permission to download (multiple) files - if so, you should grant permission.
from google.colab import files
files.download('fig7pdf.pdf')
files.download('fig7png.png')

- Now that the lateral acceleration offset is compensated, a time history of the radius $r$ of the vehicle's path can be obtained from the longitudinal velocity $v_x$ and yaw velocity $\omega_z$:

$$ r=\frac{v_x}{\omega_z}$$

- The next cell plots three graphs: the longitudinal velocity $v_x$ (integrated longitudinal acceleration of the IMU); the yaw velocity $\omega_z$ (from the IMU); and the radius $ r=\frac{v_x}{\omega_z}$.
- The dashed line in the bottom graph is the average radius calculated previously in Section 4.1.
- Note that some values of $r$ might not be plotted if there is a divide by zero condition. 

In [ ]:
ymax=0.5    # EDIT THIS TO SPECIFY THE MAXIMUM RADIUS ON THE VERTICAL AXIS (default is 0.5 m)
ymin=-0.5   # EDIT THIS TO SPECIFY THE MINIMUM RADIUS ON THE VERTICAL AXIS (default is -0.5 m)

# DO NOT CHANGE ANYTHING BELOW THIS LINE

plt.close(0) # close figure 0 if current
np.warnings.filterwarnings('ignore') # suppresses divide by zero warnings
r=np.divide(vx,omegazscaled)

fig8=plt.figure(8,figsize=(9.5,6)) 
plt.subplot(311)
plt.title('axoffset=%.2f m/s/s, omegazscaling=%.2f, average path radius (dashed)=%.4f m,' %(axoffset, omegazscaling, averagerad))
plt.ylabel(r'long. vel. $/\ \rm{m \ s^{-1}}$')
plt.plot(t,vx)
plt.subplot(312)
plt.ylabel(r'yaw vel. $/\ \rm{rad \ s^{-1}}$')
plt.plot(t,omegazscaled)
plt.subplot(313)
plt.ylim(ymin,ymax)
plt.ylabel(r'radius $/\ \rm{m}$')
plt.xlabel(r'time $t\ /\ \rm{s}$')      
plt.plot(t,r)
plt.plot(t,averagerad*np.ones_like(t),linestyle='--')
fig8.text(0.5,0.5,watermark,fontsize=70,color='gray',ha='center',va='center',rotation=45,alpha=0.15)
plt.show()

#### Q8. Comment on the whole time history of radius (solid line) in the bottom graph, and compare it to the average path radius calculated in Section 4.1 (dashed line).
> #### In particular, comment on regions where there is agreement and where there is discrepancy, and compare to the bottom graph of Figures 5 and 6.
> #### You may need to adjust the range of the vertical axis in the last graph; edit the ymax and ymin values in the cell.
> #### Remember that the encoder measures the distance travelled by the left hand rear wheel, and that the IMU is located on the longitudinal centreline of the car; what effect does this have on the calculated path radii? 

- When you have answered Question 8 run the next cell to save Figure 8 (existing files with the same name will be overwritten without warning).

In [ ]:
# Use this cell to save figure 8
if 'fig8' in globals():
  fig8.savefig("fig8pdf.pdf")
  fig8.savefig("fig8png.png")
  print('Figure 8 saved')
else: print('Error: Figure not defined')

In [ ]:
# Google colabs only: then use this cell to download figure 8
# You may get a browser pop up asking for permission to download (multiple) files - if so, you should grant permission.
from google.colab import files
files.download('fig8pdf.pdf')
files.download('fig8png.png')

- Two other combinations of the signals measured by the IMU can be used to estimate path radius, but they will not be examined in this experiment. The combinations are:
>- Integrate longitudinal acceleration to longitudinal velocity, then divide longitudinal velocity squared by lateral acceleration.
>- Divide lateral acceleration by yaw velocity squared.

## 5. Arbitrary path

- In this section you will: examine data for the vehicle travelling along an arbitrary path; calculate and plot the car's path using the measured data; generate Figure 9; and answer Question 9.
 
 
- Run the [recording/plotting cell](#record-data), and move the car along an arbitraty path.
- Ensure that the vehicle starts and finishes in exactly the same position and exactly the same angle so that you can easily check the accuracy of the calculation of the car's path using the measured data.


- As with the circular motion analysis in Section 4, there are several combinations of measurements that can be used to reconstruct the car's path.
- The wheel encoder and yaw velocity will be used, to avoid the problems associated with offset of the acceleration signals.
- The next cell calculates and plots the position of the car at every time step in ground-fixed axes X and Y, where the origin is the position of the car at the start of the run (zero time) and the ground-fixed X-axis is aligned with the vehicle-fixed longitudinal x-axis at the start of the run.
- The calculation involves determining the incremental displacement in X and Y directions at each time step.
- First, increments in displacement along the vehicle's longitudinal x-axis at each time step are calculated by using the numpy 'diff' function on the distance array from the encoder data.
- The incremental displacements are multiplied by cosine of the corresponding yaw angles (from the IMU data) at each time step to give the incremental distances in the X direction.
- The incremental displacements are multiplied by sine of the yaw angles to give the incremental distances in the Y direction.
- The incremental X and Y displacements are then added cumulatively (using the numpy 'accumulate' function) to give the X and Y coordinates at each time step.


- Run the next cell to perform this calculation and plot the path of the car.
- The start of the path is indicated with a green circle and the end of the path is indicated with a red circle.

In [12]:
#Video Cell. You may need to run this cell to load the video before you can play it
HTML('<iframe src="https://cambridgelectures.cloud.panopto.eu/Panopto/Pages/Embed.aspx?id=c26cd09f-7c61-4236-81c6-acb100b0e52c&autoplay=false&offerviewer=true&showtitle=true&showbrand=false&start=0&interactivity=all" height="405" width="720" style="border: 1px solid #464646;" allowfullscreen allow="autoplay"></iframe>')

In [ ]:
#Data load cell
[axlist, aylist, omegazlist, counterlist,tlist] = loadtxt('https://raw.githubusercontent.com/CambridgeEngineering/PartIA-Mechanics-Labs/main/IA_vehicle_motion/dataarb.csv', delimiter=',').tolist()
print('Arbitrary path data loaded')

In [ ]:
# DO NOT CHANGE ANYTHING BELOW THIS LINE

plt.close(0) # close figure 0 if current


t=np.array(tlist)   # convert lists to numpy arrays
ax=np.array(axlist)  
ay=np.array(aylist)
omegaz=np.array(omegazlist) 
counter=np.array(counterlist)

    
# calculate global coordinates at each time step
t=t-t[0] # subtract first element of the array from the whole array so that the plot starts at 0 s.
counter=counter-counter[0] # subtract first element of the array from the whole array so that the counter starts at 0.
ppr=128 # pulses per revolution of the encoder
sxcount=(counter/ppr)*(wheeldia*np.pi)
dsxcount=np.zeros_like(sxcount)
dsxcount[1:]=np.diff(sxcount)
omegazscaled=omegaz*omegazscaling
thetaz = integrate.cumtrapz(omegazscaled, t, initial=0) # integrate omegaz to find angular displacement thetaz

globalx=np.add.accumulate(np.multiply(dsxcount,np.cos(thetaz)))
globaly=np.add.accumulate(np.multiply(dsxcount,np.sin(thetaz)))
    
fig9=plt.figure(9,figsize=(9.5,6))
plt.xlabel(r'ground-fixed $X\ /\ \rm{m}$')
plt.ylabel(r'ground-fixed $Y\ /\ \rm{m}$')
plt.axis('equal')
plt.plot(globalx,globaly)
plt.plot(globalx[0],globaly[0],'go')
plt.plot(globalx[-1],globaly[-1],'ro')
fig9.text(0.5,0.5,watermark,fontsize=70,color='gray',ha='center',va='center',rotation=45,alpha=0.15)
plt.show()  

#### Q9a. Comment on Figure 9. Is it what you expected? Why?

#### Q9b. What other combinations of measured signals could be used to measure the path of the car? Would they give better or worse accuracy than the encoder and yaw velocity signals used for Figure 9? Why?

- When you have answered Question 9 run the next cell to save Figure 9 (existing files with the same name will be overwritten without warning).

In [ ]:
# Use this cell to save figure 9
if 'fig9' in globals():
  fig9.savefig("fig9pdf.pdf")
  fig9.savefig("fig9png.png")
  print('Figure 9 saved')
else: print('Error: Figure not defined')

In [ ]:
# Google colabs only: then use this cell to download figure 9
# You may get a browser pop up asking for permission to download (multiple) files - if so, you should grant permission.
from google.colab import files
files.download('fig9pdf.pdf')
files.download('fig9png.png')

## 6. Download the Figure files

- When you have saved Figures 1 to 9, you need to download them to your local machine so you can use them in your report.
- If you are using google colabs then you can do this by running the "download" cell beneath each save cell. Hopefully you have been doing this as you go through the notebook.
- If you are runing the python locally, try to identify where in your file system the notebook has been saving the files - probably in the same directory as the notebook is located. If it's not obvious, try searching your disk for "fig1pdf.pdf"
- If you cannot work out how to download the figures, you can download them individually from your browser by right clicking on them and selecting "Save Image As" (or similar depending on browser). 
- If all else fails, there is always screenshot. 

## 7. Written report

- Prepare a report of the experiment. This should take about two hours.
- A formal technical report (as undertaken in your exposition class) is NOT required.
- Instead, prepare a document that could be used as the basis of an oral presentation to an audience that is familiar with the objectives and method of the experiment, but unfamiliar with the results and conclusions.
- The report can be prepared in your lab notebook or using a computer program such as Microsoft Office, Libre Office or LateX.
- Refer to this notebook on Moodle when writing your report, to ensure that all the required Figures are included and all the questions are addressed.


- The first (title) page should state your name, college, lab group number, name of the experiment, date of the experiment, and the number of the car that you used (1 to 8).
- The main body of the report should consist of one A4 page (portrait) for each of Figures 1 to 9.
- Place the Figure in the upper half of the page, and immediately below each Figure give a Figure number (1 to 9) and an informative title.
- In the lower half of the page write a number of bullet points that describe the Figure and the conclusions that can be drawn from it (imagine that these bullet points would form the basis of an oral presentation).
- Include in your bullet points the answers to Questions 1 to 9.
- The last (conclusion) page should state the main conclusions as bullet points.
- The bullet points should be concise (phrases or short sentences, not paragraphs!)
- Remember that the report should be written individually and not in collaboration with your lab partner.


- Sign up as a lab group for a marking session to take place within 15 days of performing the experiment, one 15min time slot per lab group.
- Sign up is on the moodle site: https://www.vle.cam.ac.uk/course/view.php?id=217431
- It is recommended that you sign up on the day of the experiment; don't wait until you have finished writing the report since there may be no suitable slots remaining.
- Sign up at least 24hours before the session otherwise the marker will not attend.
- Marking takes place on teams - look on the modle site to find the right link.
- Submit your report electronically on the moodle site before your marking session.

## Appendix - Links to further information about the equipment

Inertial measurement unit:

https://www.bosch-sensortec.com/bst/products/all_products/bno055

https://learn.adafruit.com/adafruit-bno055-absolute-orientation-sensor/overview

Rotary optical encoder:

https://www.bourns.com/docs/default-document-library/enc1j.pdf?sfvrsn=2c0f9df1_0

https://playground.arduino.cc/Main/RotaryEncoders

Arduino:

https://www.arduino.cc/